<a href="https://colab.research.google.com/github/minhyeok2487/ML_DL_Practice/blob/main/%EC%99%80%EC%9D%B8_%ED%92%88%EC%A7%88_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 호출

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns    

In [ ]:
import os
os.chdir("/content/drive/MyDrive/데이터 분석/[DACON] 와인 품질 분류/data")
print(os.getcwd())

# 2. 데이터 로드

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()
#index 제외 13개의 변수

In [ ]:
test.head()
# 종속변수 - quality

In [ ]:
train.info()
# type만 오브젝트, 결측치 없음

In [ ]:
# train의 변수 간 상관관계를 살펴봅니다.
plt.figure(figsize=(12,12))
sns.heatmap(data = train.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
# test의 변수 간 상관관계를 살펴봅니다.
plt.figure(figsize=(12,12))
sns.heatmap(data = test.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='gray')

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,13):
    plt.subplot(3,4,i)
    sns.distplot(train.iloc[:,i])
plt.tight_layout()
plt.show()

# 3. 데이터 전처리

## 1) Type 변수 확인

In [ ]:
train['type'].value_counts()

In [ ]:
# 각각 0, 1로 변환
train['type'] = train['type'].map({'white':0, 'red':1}).astype(int)
test['type'] = test['type'].map({'white':0, 'red':1}).astype(int)

In [ ]:
train.head()

## 2) 데이터 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_df = train.copy()
train_df = train_df.drop(['index','quality','type'], axis=1)
train_df

In [ ]:
transformer = MinMaxScaler()
train_df[:] = transformer.fit_transform(train_df[:])
train_df

In [ ]:
test_df = test.copy()
test_df = test_df.drop(['index','type'], axis=1)
test_df[:] = transformer.fit_transform(test_df[:])
test_df

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,10):
    plt.subplot(3,4,i)
    sns.distplot(train_df.iloc[:,i])
plt.tight_layout()
plt.show()

# 3. 간단한 모델 학습

## 1) 데이터 정형화

In [ ]:
train_x = train_df
train_x['type'] = train['type'].values 
train_x

In [ ]:
train_y = train['quality']
train_y

In [ ]:
test_x = test_df
test_x['type'] = test['type'].values 
test_x

## 2) lightgbm 모델 학습 
**점수 : 0.497**

In [ ]:
train_x = train.drop(['index', 'quality'], axis = 1)
train_y = train['quality']
test_x = test.drop('index', axis = 1)

In [ ]:
train_x

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.3, 
                                                    random_state = 0, stratify=train_y)

In [ ]:
import lightgbm as lgbm
model = lgbm.LGBMClassifier()
model.fit(X_train,y_train)

## 3) 예측 데이터 생성

In [ ]:
y_pred = model.predict(X_test)
y_pred

## 4) View Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

0.6242

## 4) 제출파일

In [ ]:
smpl_sub = pd.read_csv('sample_submission.csv')
smpl_sub['quality'] = y_pred

In [ ]:
smpl_sub

In [ ]:
smpl_sub.to_csv('submission_210821.csv', index=False)

# 4. KNN 모델 학습 

In [ ]:
train_x = train.drop(['index', 'quality'], axis = 1)
train_y = train['quality']
test_x = test.drop('index', axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.3, 
                                                    random_state = 0, stratify=train_y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
  knn.fit(X_train, y_train)

In [ ]:
print(knn.score(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
k_list = range(1,101)
accuracies = []
for k in k_list:
  classifier = KNeighborsClassifier(n_neighbors = k)
  classifier.fit(X_train, y_train)
  accuracies.append(classifier.score(X_test, y_test))
plt.plot(k_list, accuracies)
plt.xlabel("k")
plt.ylabel("Validation Accuracy")
plt.title("Breast Cancer Classifier Accuracy")
plt.show()